In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
# from huggingface_hub import notebook_login

# notebook_login()

## Imports

In [4]:
from datasets import load_dataset, get_dataset_config_names, load_metric

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding

## Load data

In [5]:
dataset_name = "amazon_reviews_multi"
langs = get_dataset_config_names(dataset_name)
langs

['all_languages', 'de', 'en', 'es', 'fr', 'ja', 'zh']

In [6]:
english_dataset = load_dataset("amazon_reviews_multi", "en")
english_dataset

Reusing dataset amazon_reviews_multi (/data/.cache/hf/datasets/amazon_reviews_multi/en/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
})

In [7]:
japanese_dataset = load_dataset("amazon_reviews_multi", "ja")
japanese_dataset

Reusing dataset amazon_reviews_multi (/data/.cache/hf/datasets/amazon_reviews_multi/ja/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
})

In [8]:
japanese_dataset["train"][0]

{'review_id': 'ja_0388536',
 'product_id': 'product_ja_0270003',
 'reviewer_id': 'reviewer_ja_0454098',
 'stars': 1,
 'review_body': '普段使いとバイクに乗るときのブーツ兼用として購入しました。見た目や履き心地は良いです。 しかし、２ヶ月履いたらゴム底が削れて無くなりました。また、バイクのシフトペダルとの摩擦で表皮が剥がれ、本革でないことが露呈しました。ちなみに防水とも書いていますが、雨の日は内部に水が染みます。 安くて見た目も良く、履きやすかったのですが、耐久性のなさ、本革でも防水でも無かったことが残念です。結局、本革の防水ブーツを買い直しました。',
 'review_title': '本革でも防水でもない',
 'language': 'ja',
 'product_category': 'shoes'}

In [9]:
japanese_dataset.set_format("pandas")

In [10]:
japanese_df = japanese_dataset["train"][:]
japanese_df.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,ja_0388536,product_ja_0270003,reviewer_ja_0454098,1,普段使いとバイクに乗るときのブーツ兼用として購入しました。見た目や履き心地は良いです。 しか...,本革でも防水でもない,ja,shoes
1,ja_0701307,product_ja_0540923,reviewer_ja_0635293,1,十分な在庫を用意できない販売元も悪いですが、Amazonやら楽⚪︎が転売を認めちゃってるのが...,Amazonが転売を認めてるから…,ja,video_games
2,ja_0724445,product_ja_0113803,reviewer_ja_0598011,1,見た目はかなりおしゃれで気に入りました。2、3回持ち歩いた後いつも通りゼンマイを巻いていたら...,時計として使えません。,ja,watch
3,ja_0018251,product_ja_0063026,reviewer_ja_0949142,1,よくある部分での断線はしませんでした ただiphoneとの接続部で接触不良、折れました ip...,安かろう、悪かろう,ja,pc
4,ja_0727421,product_ja_0989267,reviewer_ja_0163994,1,プラモデルの塗装剥離に使う為に購入 届いて早速使ってみた 結果 １ヶ月経っても未だに剥離出来...,元々使えなかったのか？,ja,home_improvement


In [11]:
japanese_df["product_category"].value_counts()

home                      18662
wireless                  16333
sports                    15788
pc                        14892
kitchen                   13356
automotive                13124
drugstore                 10286
electronics                9867
shoes                      8924
toy                        8605
beauty                     8476
apparel                    8166
home_improvement           6717
grocery                    6409
other                      5826
book                       5591
pet_products               4842
camera                     4415
office_product             4257
video_games                2559
digital_video_download     2506
digital_ebook_purchase     2218
industrial_supplies        2065
baby_product               1930
watch                      1801
musical_instruments        1218
jewelry                    1138
luggage                      11
furniture                     9
lawn_and_garden               9
Name: product_category, dtype: int64

In [12]:
japanese_df["stars"].value_counts()

1    40000
2    40000
3    40000
4    40000
5    40000
Name: stars, dtype: int64

In [13]:
japanese_dataset.reset_format()
toys_dataset = japanese_dataset.filter(lambda x : x["product_category"] == "toy")
toys_dataset

Loading cached processed dataset at /data/.cache/hf/datasets/amazon_reviews_multi/ja/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609/cache-7081981fac9628f1.arrow
Loading cached processed dataset at /data/.cache/hf/datasets/amazon_reviews_multi/ja/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609/cache-dd820ccda07b6122.arrow
Loading cached processed dataset at /data/.cache/hf/datasets/amazon_reviews_multi/ja/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609/cache-6e1b8acccf00b765.arrow


DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 8605
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 240
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 224
    })
})

In [16]:
toys_dataset = japanese_dataset.filter(lambda x : x["stars"] != 3)

Loading cached processed dataset at /data/.cache/hf/datasets/amazon_reviews_multi/ja/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609/cache-e726086222a10b59.arrow
Loading cached processed dataset at /data/.cache/hf/datasets/amazon_reviews_multi/ja/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609/cache-078818e738c6acaa.arrow
Loading cached processed dataset at /data/.cache/hf/datasets/amazon_reviews_multi/ja/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609/cache-d3c487c2c3cb4e48.arrow


In [17]:
def merge_star_ratings(examples):
    if examples["stars"] <= 2:
        label = 0
    else:
        label = 1
    return {"labels": label}

In [18]:
toys_dataset = toys_dataset.map(merge_star_ratings)

Loading cached processed dataset at /data/.cache/hf/datasets/amazon_reviews_multi/ja/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609/cache-c6ae65fa62e7c89d.arrow
Loading cached processed dataset at /data/.cache/hf/datasets/amazon_reviews_multi/ja/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609/cache-cf2b5f2337c317b9.arrow
Loading cached processed dataset at /data/.cache/hf/datasets/amazon_reviews_multi/ja/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609/cache-4a57b7cb571ef255.arrow


In [21]:
toys_dataset

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category', 'labels'],
        num_rows: 160000
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category', 'labels'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category', 'labels'],
        num_rows: 4000
    })
})

## Preprocessing

In [22]:
model_checkpoint = "cl-tohoku/bert-base-japanese-char-v2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [23]:
text = "こんにちは、私の名前はルイスです。"

In [24]:
text_encoded = tokenizer(text)
text_encoded

{'input_ids': [2, 869, 933, 893, 883, 897, 828, 3946, 896, 1564, 1402, 897, 1013, 942, 963, 889, 875, 829, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [25]:
tokenizer.convert_ids_to_tokens(text_encoded["input_ids"])

['[CLS]',
 'こ',
 'ん',
 'に',
 'ち',
 'は',
 '、',
 '私',
 'の',
 '名',
 '前',
 'は',
 'ル',
 'イ',
 'ス',
 'で',
 'す',
 '。',
 '[SEP]']

In [26]:
def tokenize_reviews(examples):
    return tokenizer(examples["review_body"], truncation=True, max_length=512)

In [27]:
columns = toys_dataset["train"].column_names
columns.remove("labels")
tokenized_datasets = toys_dataset.map(tokenize_reviews, batched=True, remove_columns=columns)

Loading cached processed dataset at /data/.cache/hf/datasets/amazon_reviews_multi/ja/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609/cache-f2277e2ce74fbf7d.arrow
Loading cached processed dataset at /data/.cache/hf/datasets/amazon_reviews_multi/ja/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609/cache-5d903108055545e5.arrow
Loading cached processed dataset at /data/.cache/hf/datasets/amazon_reviews_multi/ja/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609/cache-6131df334c9ee9fd.arrow


## Load model

In [48]:
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

loading configuration file https://huggingface.co/cl-tohoku/bert-base-japanese-char-v2/resolve/main/config.json from cache at /data/.cache/hf/transformers/728fe79cdd383106d018c4605d507204ce59ad8cae2b3648479565fae88cce7f.2b02fe9ca86e2a6935ff2e4bff00bf213aeb54f4de4adf939403dd913da5fdf1
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertJapaneseTokenizer",
  "transformers_version": "4.11.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 6144
}

loading weights file https://huggingface.co/cl-tohoku/bert-base-japanese-char-v2/res

In [49]:
text_encoded = tokenizer(text, return_tensors="pt")

In [50]:
text_encoded["input_ids"]

tensor([[   2,  869,  933,  893,  883,  897,  828, 3946,  896, 1564, 1402,  897,
         1013,  942,  963,  889,  875,  829,    3]])

In [51]:
model(**text_encoded)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.0028, -0.4409]], grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)

## Load metrics

In [52]:
accuracy = load_metric("accuracy")

## Train!

In [53]:
batch_size = 16
num_train_epochs = 3
model_name = model_checkpoint.split("/")[-1]
num_train_samples = 20_000
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(num_train_samples))
logging_steps = len(train_dataset) // (2 * batch_size * num_train_epochs)

training_args = TrainingArguments(
    "results",
    overwrite_output_dir=True,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    num_train_epochs=num_train_epochs,
    logging_steps=logging_steps,
    push_to_hub=True,
    hub_model_id=f"lewtun/{model_name}-finetuned-amazon-jap",
)

Loading cached shuffled indices for dataset at /data/.cache/hf/datasets/amazon_reviews_multi/ja/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609/cache-d18e4bc70a19c2db.arrow
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [54]:
import numpy as np

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

In [55]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [56]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'token_type_ids'],
        num_rows: 160000
    })
    validation: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'token_type_ids'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'token_type_ids'],
        num_rows: 4000
    })
})

In [57]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=train_dataset,
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/home/lewis/git/workshops/machine-learning-tokyo/results is already a clone of https://huggingface.co/lewtun/bert-base-japanese-char-v2-finetuned-amazon-jap. Make sure you pull the latest changes with `repo.git_pull()`.


In [47]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 4000
  Batch size = 16


{'eval_loss': 0.2563161849975586,
 'eval_accuracy': 0.90175,
 'eval_runtime': 25.6708,
 'eval_samples_per_second': 155.819,
 'eval_steps_per_second': 9.739}

In [58]:
trainer.train()

***** Running training *****
  Num examples = 20000
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3750


Epoch,Training Loss,Validation Loss,Accuracy
1,0.252800,0.213693,0.919750
2,0.178600,0.242306,0.926000
3,0.100400,0.321566,0.924750


Saving model checkpoint to results/checkpoint-500
Configuration saved in results/checkpoint-500/config.json
Model weights saved in results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in results/checkpoint-500/tokenizer_config.json
Special tokens file saved in results/checkpoint-500/special_tokens_map.json
tokenizer config file saved in results/tokenizer_config.json
Special tokens file saved in results/special_tokens_map.json
Saving model checkpoint to results/checkpoint-1000
Configuration saved in results/checkpoint-1000/config.json
Model weights saved in results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in results/checkpoint-1000/special_tokens_map.json
tokenizer config file saved in results/tokenizer_config.json
Special tokens file saved in results/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 4000
  Batch size = 16
Saving model checkpoint to resul

TrainOutput(global_step=3750, training_loss=0.18888962638775508, metrics={'train_runtime': 1344.178, 'train_samples_per_second': 44.637, 'train_steps_per_second': 2.79, 'total_flos': 9311138471489280.0, 'train_loss': 0.18888962638775508, 'epoch': 3.0})